# 🚀 Ai_wolf (Wolf_V5) 專案 Colab 快速啟動器 (超穩健版)

本 Notebook 用於在 Google Colaboratory 環境中一鍵部署和啟動 Ai_wolf (Wolf_V5) 的後端 FastAPI 服務和前端 Streamlit 應用程式。

**特點**:
*   **強制環境清理**：每次執行都會清理舊的專案檔案，確保從乾淨的狀態開始。
*   **自動化依賴安裝**：從根目錄的 `requirements.txt` 安裝所有必要的 Python 套件。
*   **最新程式碼部署**：自動從 GitHub 克隆最新的專案程式碼。
*   **雙服務啟動**：同時啟動背景 FastAPI 服務 (port 8000) 和前景 Streamlit 服務 (port 8501)。
*   **日誌分離與查看**：FastAPI 的日誌會寫入指定的日誌檔案，Streamlit 的日誌會直接顯示。提供一個額外的儲存格來查看 FastAPI 的日誌。
*   **直接錯誤顯示 (Streamlit)**：Streamlit 應用程式的任何啟動錯誤會直接顯示在主執行儲存格中。
*   **公開網址提供 (Streamlit)**：透過 Colab 內建代理或 Cloudflared 提供一個公開的 URL 來訪問 Streamlit 應用程式。

**首次使用設定**:
1.  **API 金鑰**:
    *   在 Colab 的左側邊欄找到鑰匙圖示 (Secrets)。
    *   新增兩個 Secret：
        *   `GOOGLE_API_KEY`：填入您的 Google Gemini API 金鑰。
        *   `FRED_API_KEY`：填入您的 FRED API 金鑰。
    *   確保 "Notebook access" 已啟用。應用程式 (`app.py` 和後端服務) 會優先從這裡讀取金鑰。
2.  **Google Drive 授權**:
    *   執行主 Notebook 儲存格時，會提示您授權 Google Drive 存取。這是為了將專案檔案和日誌儲存在您的 Drive 中，以便持久化。
    *   專案將被下載到 `/content/drive/MyDrive/wolfAI`。
    *   應用程式日誌將寫入 `/content/drive/MyDrive/Wolf_Data/logs/` (包含 `fastapi.log` 和 `streamlit.log`)。

**執行步驟**:
1.  點擊下方第一個 Code Cell (標題為 "🚀 一鍵啟動 Wolf_V5 (FastAPI + Streamlit)") 左側的「執行」按鈕。
2.  耐心等待所有階段完成（環境設定、套件安裝、專案下載、FastAPI 啟動、Streamlit 啟動）。
3.  FastAPI 服務會在背景啟動，其日誌路徑會被打印出來。
4.  Streamlit 服務成功啟動後，會顯示一個**綠色的按鈕**，點擊即可在新分頁中打開 Ai_wolf Streamlit 應用程式。
5.  主 Code Cell 的輸出會持續顯示來自 Streamlit 應用程式的即時日誌和任何錯誤訊息。
6.  執行完成後，您可以執行第二個 Code Cell (標題為 "📄 查看 FastAPI 日誌") 來查看 FastAPI 服務的最新日誌。

### 方法三：一鍵綜合健康度檢測報告

此方法提供一個快速的方式來檢查您的 Colab 環境、API 金鑰設定、以及 Wolf V5 應用程式是否能成功與外部服務 (如 Google Gemini, FRED) 連線。

**使用步驟:**
1. **複製腳本**：請前往專案的 `README.md` 文件，找到「綜合健康度檢測腳本」部分，並完整複製提供的 Python 腳本。
2. **貼上腳本**：在本 Colab Notebook 的最後一個 Code Cell (標題為 "⚙️ 貼上「綜合健康度檢測腳本」於此處並執行") 中，貼上您剛剛複製的腳本。
3. **執行檢測**：執行該 Code Cell。
4. **查看報告**：腳本會輸出一份詳細的檢測報告，幫助您判斷各個環節是否正常運作。

如果報告中顯示任何錯誤或警告，請依照錯誤訊息的提示進行調整（例如：檢查 API 金鑰是否正確設定、Colab Secret 是否啟用等）。

**注意**:
*   如果遇到任何問題，請仔細檢查 Colab 儲存格中的輸出訊息，特別是 FastAPI 和 Streamlit 的日誌。
*   要停止應用程式，可以點擊主 Code Cell 執行按鈕旁邊的「停止」按鈕。這會嘗試終止 Streamlit 和 FastAPI 服務。

In [ ]:
# === Wolf_V5 專案：環境設置、部署、啟動與即時監控 (FastAPI + Streamlit) ===
# 此版本啟動 FastAPI 後端服務，並直接捕獲 Streamlit 的所有輸出以顯示錯誤。

import os
import subprocess
import time
import threading
import sys # Added for sys.executable
import signal # Added for signal handling
from IPython.display import display, HTML, clear_output
# google.colab.drive 和 google.colab.output 只在 Colab 中可用
try:
    from google.colab import drive
    from google.colab.output import eval_js
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    # 如果需要在本地模擬這些行為，可以在這裡定義 mock 函數或類別
    def mock_drive_mount(path, force_remount=False): print(f"Mock: Drive mount requested for {path}")
    drive = type('Drive', (object,), {'mount': mock_drive_mount})()
    def mock_eval_js(code): print(f"Mock: eval_js called with {code}"); return "http://localhost:8501" # 返回一個模擬 URL
    eval_js = mock_eval_js
    print("非 Colab 環境，部分 Colab 特定功能將被模擬或跳過。")


# --- 階段一：環境設置與專案部署 ---
print("--- 階段一：環境設置與專案部署 ---")
print("\n[1/5] 正在安裝必要的 Python 套件 (將從 requirements.txt 安裝)...")

GDRIVE_PROJECT_DIR = "/content/drive/MyDrive/wolfAI" 
LOCAL_PROJECT_DIR = "/content/wolfAI" 
PROJECT_DIR_TO_USE = None 

if IN_COLAB:
    print("\n[2/5] 正在嘗試掛載 Google Drive...")
    try:
        drive.mount('/content/drive', force_remount=True)
        print("✅ Google Drive 掛載成功！")
        PROJECT_DIR_TO_USE = GDRIVE_PROJECT_DIR

        WOLF_DATA_BASE_GDRIVE = "/content/drive/MyDrive/Wolf_Data"
        print(f"    檢查/創建 Wolf_Data 基礎目錄: {WOLF_DATA_BASE_GDRIVE}")
        if not os.path.exists(WOLF_DATA_BASE_GDRIVE):
            os.makedirs(WOLF_DATA_BASE_GDRIVE, exist_ok=True)
            print(f"    ✅ '{WOLF_DATA_BASE_GDRIVE}' 已創建。")
        else:
            print(f"    ➡️ '{WOLF_DATA_BASE_GDRIVE}' 已存在。")

        sub_dirs_to_create = ["source_documents/masters", "source_documents/shan_jia_lang", "weekly_reports", "logs"]
        for sub_dir in sub_dirs_to_create:
            full_sub_dir_path = os.path.join(WOLF_DATA_BASE_GDRIVE, sub_dir)
            print(f"    檢查/創建子目錄: {full_sub_dir_path}")
            if not os.path.exists(full_sub_dir_path):
                os.makedirs(full_sub_dir_path, exist_ok=True)
                print(f"    ✅ '{full_sub_dir_path}' 已創建。")
            else:
                print(f"    ➡️ '{full_sub_dir_path}' 已存在。")

    except Exception as e:
        print(f"⚠️ Google Drive 掛載失敗: {e}")
        print("將嘗試使用 Colab 本地儲存空間。如果重新執行，之前在 Drive 中的資料可能不會被使用。")
        PROJECT_DIR_TO_USE = LOCAL_PROJECT_DIR
else:
    print("\n[2/5] 跳過 Google Drive 掛載 (非 Colab 環境)。")
    PROJECT_DIR_TO_USE = LOCAL_PROJECT_DIR 

WOLF_DATA_GDRIVE_BASE_INFO = "/content/drive/MyDrive/Wolf_Data" 
WOLF_DATA_LOCAL_BASE_INFO = "/content/Wolf_Data" 
APP_EXPECTED_LOG_DIR = None 

if PROJECT_DIR_TO_USE == GDRIVE_PROJECT_DIR: 
    APP_EXPECTED_LOG_DIR = os.path.join(WOLF_DATA_GDRIVE_BASE_INFO, "logs")
    if not os.path.exists(WOLF_DATA_GDRIVE_BASE_INFO):
        print(f"    ⚠️ Sanity Check: Wolf_Data base {WOLF_DATA_GDRIVE_BASE_INFO} not found, creating.")
        os.makedirs(WOLF_DATA_GDRIVE_BASE_INFO, exist_ok=True)
else: 
    APP_EXPECTED_LOG_DIR = os.path.join(WOLF_DATA_LOCAL_BASE_INFO, "logs")
    if not os.path.exists(WOLF_DATA_LOCAL_BASE_INFO): 
        os.makedirs(WOLF_DATA_LOCAL_BASE_INFO, exist_ok=True)
        print(f"    ✅ 已確認/建立模擬的本地 Wolf_Data 基礎目錄: {WOLF_DATA_LOCAL_BASE_INFO}")

# Ensure the APP_EXPECTED_LOG_DIR itself exists
if not os.path.exists(APP_EXPECTED_LOG_DIR):
    os.makedirs(APP_EXPECTED_LOG_DIR, exist_ok=True)
    print(f"    ✅ Log directory {APP_EXPECTED_LOG_DIR} created.")
else:
    print(f"    ➡️ Log directory {APP_EXPECTED_LOG_DIR} already exists.")

FASTAPI_LOG_PATH = os.path.join(APP_EXPECTED_LOG_DIR, "fastapi.log")
STREAMLIT_LOG_PATH = os.path.join(APP_EXPECTED_LOG_DIR, "streamlit.log") # For consistency, though Streamlit output is usually to stdout

print(f"\n[3/5] 設定專案路徑: {PROJECT_DIR_TO_USE}")
print(f"    應用程式資料/日誌目錄預期在: {APP_EXPECTED_LOG_DIR}")
print(f"    FastAPI 日誌將儲存於: {FASTAPI_LOG_PATH}")

if os.path.exists(PROJECT_DIR_TO_USE):
    print(f"    偵測到已存在的專案目錄 '{PROJECT_DIR_TO_USE}'。執行強制清理...")
    try:
        import shutil
        shutil.rmtree(PROJECT_DIR_TO_USE)
        print(f"    ✅ '{PROJECT_DIR_TO_USE}' 已成功刪除。")
    except Exception as e_rm:
        print(f"    ❌ 刪除 '{PROJECT_DIR_TO_USE}' 時發生錯誤: {e_rm}")
        print("       請手動檢查並刪除該資料夾，然後重新執行此儲存格。")
        raise 
else:
    print(f"    '{PROJECT_DIR_TO_USE}' 不存在，無需清理。")

project_parent_dir = os.path.dirname(PROJECT_DIR_TO_USE)
if not os.path.exists(project_parent_dir) and PROJECT_DIR_TO_USE != LOCAL_PROJECT_DIR: 
     os.makedirs(project_parent_dir, exist_ok=True) 

STREAMLIT_APP_PATH = os.path.join(PROJECT_DIR_TO_USE, "app.py")
REQUIREMENTS_PATH = os.path.join(PROJECT_DIR_TO_USE, "requirements.txt")
STREAMLIT_PORT = 8501 
FASTAPI_PORT = 8000

print(f"✅ 已確認/建立專案目錄。") 
print(f"\n[4/5] 正在從 GitHub 獲取最新程式碼...")
GIT_REPO_URL = "https://github.com/hsp1234-web/Ai_wolf.git" 
git_clone_command = ["git", "clone", "--depth", "1", GIT_REPO_URL, PROJECT_DIR_TO_USE]
result = subprocess.run(git_clone_command, capture_output=True, text=True)
if result.returncode == 0:
    print("    ✅ 專案克隆完成！")
else:
    print(f"    ❌ 專案克隆失敗:")
    print(result.stdout)
    print(result.stderr)
    raise SystemExit("Git clone 失敗，終止執行。")

if os.path.isfile(REQUIREMENTS_PATH):
    print("    正在從 requirements.txt 安裝所有依賴...")
    pip_install_command = ["pip", "install", "-q", "-r", REQUIREMENTS_PATH]
    result_pip = subprocess.run(pip_install_command, capture_output=True, text=True)
    if result_pip.returncode == 0:
        print("    ✅ 依賴安裝完成！")
    else:
        print(f"    ❌ 依賴安裝失敗:")
        print(result_pip.stdout) 
        print(result_pip.stderr)
else:
    print(f"    ⚠️ 警告: 未找到 requirements.txt 於 {REQUIREMENTS_PATH}。跳過依賴安裝步驟。")

print(f"\n[5/5] 檢查主要應用程式檔案...")
if not os.path.isfile(STREAMLIT_APP_PATH):
    raise SystemExit(f"❌ 錯誤：未在 {PROJECT_DIR_TO_USE} 中找到 app.py！")
print(f"✅ 成功找到主要應用程式檔案: {STREAMLIT_APP_PATH}")
print("\n--- 環境設置與專案部署階段完成 ---\n")
time.sleep(1)
if IN_COLAB: clear_output(wait=True)

# --- 階段二：啟動後端 FastAPI 服務 ---
print("\n--- 階段二：啟動後端 FastAPI 服務 ---")
FASTAPI_COMMAND = f"nohup python -m uvicorn backend.main:app --host 0.0.0.0 --port {FASTAPI_PORT} --app-dir {PROJECT_DIR_TO_USE} > {FASTAPI_LOG_PATH} 2>&1 &"
print(f"[1/2] 準備啟動 FastAPI 服務...")
print(f"    執行命令: {FASTAPI_COMMAND}")
fastapi_process_group_launcher = None # Renamed for clarity
try:
    fastapi_process_group_launcher = subprocess.Popen(FASTAPI_COMMAND, shell=True, preexec_fn=os.setpgrp)
    print(f"    ✅ FastAPI 啟動指令已送出。日誌將寫入 {FASTAPI_LOG_PATH}")
    print(f"    ⏳ 等待約 10 秒讓 FastAPI 完成初始化...")
    time.sleep(10)
    if os.path.exists(FASTAPI_LOG_PATH) and os.path.getsize(FASTAPI_LOG_PATH) > 0:
        print(f"    👍 FastAPI 日誌檔案已創建並有內容。可執行下一個 Cell 查看日誌。")
    elif os.path.exists(FASTAPI_LOG_PATH):
        print(f"    🤔 FastAPI 日誌檔案已創建但目前為空。服務可能仍在啟動中或遇到問題。請檢查日誌。")
    else:
        print(f"    ⚠️ FastAPI 日誌檔案 ({FASTAPI_LOG_PATH}) 未創建。服務可能啟動失敗。請檢查 Colab Runtime Logs。")
except Exception as e_fastapi:
    print(f"    ❌ 啟動 FastAPI 時發生錯誤: {e_fastapi}")
    print(f"       請檢查日誌 {FASTAPI_LOG_PATH} (如果存在) 以及 Colab 的 Runtime Logs。")

# --- 階段三：啟動 Streamlit 並直接監控其輸出 ---
print("\n--- 階段三：啟動 Streamlit 前端服務 ---")
streamlit_process = None
monitor_thread = None

# --- Helper Functions for URL Fetching and Display (for Streamlit) ---
EVAL_JS_MAX_RETRIES = 5
EVAL_JS_RETRY_DELAY = 5
CLOUDFLARED_TIMEOUT_SECONDS = 30

def fetch_url_with_eval_js(port_num, max_retries_eval, retry_delay_eval, eval_js_func, time_module, print_func, in_colab_flag):
    if not in_colab_flag:
        print_func("   eval_js: Not in Colab, skipping this method.")
        return None
    
    local_tunnel_url = None
    print_func(f"\n🔗 [URL Fetch Method 1: Colab eval_js for Port {port_num}] 開始嘗試獲取 Streamlit 網址...")
    for attempt in range(max_retries_eval):
        print_func(f"   Attempting to fetch URL via eval_js (try {attempt + 1}/{max_retries_eval} for port {port_num})...")
        try:
            proxy_url_eval = eval_js_func(f'google.colab.kernel.proxyPort({port_num})', timeout_sec=10)
            if proxy_url_eval:
                local_tunnel_url = proxy_url_eval
                print_func(f"   SUCCESS: eval_js returned URL for port {port_num}: {local_tunnel_url}")
                break
            else:
                print_func(f"   INFO: eval_js attempt {attempt + 1} returned no URL.")
        except NameError: 
            print_func("   WARNING: eval_js attempt NameError (eval_js not defined). Cannot use this method.")
            break 
        except Exception as e:
            print_func(f"   WARNING: eval_js attempt {attempt + 1} failed with error: {e}")
            if 'proxyPort is not defined' in str(e) or 'google.colab.kernel.proxyPort is not a function' in str(e):
                print_func(f"   INFO: Colab Kernel ProxyPort feature seems unavailable at this moment.")
        if attempt < max_retries_eval - 1:
            print_func(f"   Retrying in {retry_delay_eval} seconds...")
            time_module.sleep(retry_delay_eval)
        else:
            print_func(f"   ❌ eval_js: Reached max retries ({max_retries_eval}), failed to get URL for port {port_num}.")
    return local_tunnel_url

def fetch_url_with_cloudflared(port_num, timeout_seconds_cf, subprocess_mod, os_mod, threading_mod, time_mod, print_func):
    print_func(f"\n🔗 [URL Fetch Method 2: cloudflared for Port {port_num}] Attempting to fetch Streamlit URL via cloudflared...")
    tunnel_url_cf_holder = [None] 
    cloudflared_process_cf = None 
    try:
        subprocess_mod.run(["cloudflared", "--version"], check=True, capture_output=True)
        print_func("    cloudflared 已安裝。")
    except (subprocess_mod.CalledProcessError, FileNotFoundError):
        print_func("    cloudflared 未安裝或未在 PATH 中。正在嘗試安裝...")
        npm_install_cmd = ["npm", "install", "-g", "cloudflared"]
        npm_result = subprocess_mod.run(npm_install_cmd, capture_output=True, text=True)
        if npm_result.returncode == 0:
            print_func("    ✅ cloudflared 安裝成功 (透過 npm)。")
        else:
            print_func(f"    ❌ cloudflared (npm) 安裝失敗: {npm_result.stderr}")
            return None 

    cloudflared_process_cf = subprocess_mod.Popen(
        ['cloudflared', 'tunnel', '--url', f'http://localhost:{port_num}'],
        stdout=subprocess_mod.PIPE, stderr=subprocess_mod.PIPE, text=True, encoding='utf-8'
    )

    def get_cf_url_from_stderr(stderr_stream, url_holder_list, proc):
        for line in iter(stderr_stream.readline, ''):
            print_func(f"[Cloudflared] {line}", end='', flush=True)
            if ".trycloudflare.com" in line:
                extracted_url = line[line.find("https://"):].split(" ")[0]
                url_holder_list[0] = extracted_url
                print_func(f"    SUCCESS: cloudflared thread found URL: {url_holder_list[0]}")
                break 
            if proc and proc.poll() is not None: # Stop if process died
                break
        stderr_stream.close()

    cf_thread = threading_mod.Thread(target=get_cf_url_from_stderr, args=(cloudflared_process_cf.stderr, tunnel_url_cf_holder, cloudflared_process_cf))
    cf_thread.daemon = True
    cf_thread.start()

    start_time = time_mod.time()
    while not tunnel_url_cf_holder[0] and (time_mod.time() - start_time) < timeout_seconds_cf:
        if cloudflared_process_cf.poll() is not None: 
            print_func("   WARNING: cloudflared process terminated unexpectedly.")
            break
        time_mod.sleep(1)
    
    if not tunnel_url_cf_holder[0]:
        print_func(f"   WARNING: cloudflared timed out or failed to provide a URL for port {port_num}. Check cloudflared logs.")
        if cloudflared_process_cf.poll() is None: 
             try:
                cloudflared_process_cf.terminate()
                cloudflared_process_cf.wait(timeout=5)
             except Exception:
                cloudflared_process_cf.kill()
    else:
        print_func(f"   INFO: cloudflared setup for port {port_num} appears complete (URL may have been found).")
    
    return tunnel_url_cf_holder[0]

SUCCESS_HTML_TEMPLATE_STR = """
    <div style='border: 2px solid #4CAF50; padding: 20px; border-radius: 10px;'>
        <h2 style='color: #2E7D32;'>🎉 Streamlit 前端已成功啟動！</h2>
        <p>FastAPI 後端服務應在背景運行 (日誌位於 {fastapi_log_path})。</p>
        <p>Streamlit 日誌和錯誤將直接顯示在本儲存格的下方。</p>
        <p><a href='{url}' target='_blank' style='padding:12px 25px; background-color:#4CAF50; color:white; text-decoration:none; border-radius:8px;'>
           🚀 點此開啟 Wolf_V5 Streamlit 應用程式
        </a></p>
        <p style='font-size:0.8em; color:grey;'>如果無法訪問 Streamlit，請檢查下方 Streamlit 和 Cloudflared (如果使用) 的輸出日誌。</p>
    </div>
"""

FAILURE_HTML_TEMPLATE_STR = """
    <div style='border: 2px solid #F44336; padding: 20px; border-radius: 10px; background-color: #fff0f0;'>
        <h2 style='color: #C62828;'>❌ 自動獲取 Streamlit 應用程式訪問連結失敗</h2>
        <p style='color:#D32F2F;'>經過多次嘗試，腳本未能自動獲取到一個可公開訪問的 Streamlit URL。</p>
        <p>FastAPI 後端服務可能仍在背景運行 (日誌位於 {fastapi_log_path})。請檢查其日誌確認狀態。</p>
        <p style='margin-top:15px;'><strong>可能原因及排查建議 (Streamlit)：</strong></p>
        <ul style='text-align:left; margin-left: 20px;'>
            <li><strong>Streamlit 服務未成功啟動：</strong>請向上回顧本儲存格的日誌輸出，特別是 <code>[Streamlit]</code> 開頭的行。</li>
            <li><strong>Colab 環境問題 (proxyPort/Cloudflared)：</strong>檢查相關日誌。</li>
        </ul>
        <p style='font-size:0.9em; color:gray; margin-top:20px;'>請注意：Streamlit 應用程式本身可能仍在後台運行，即使此處未能生成直接訪問按鈕。</p>
    </div>
"""

def display_result_html(url_to_display, success_template, failure_template, display_func, HTML_class, clear_output_func, in_colab_flag, print_func, fastapi_log_path_for_template):
    print_func("\n--- Preparing to display Streamlit access link/message ---")
    # Ensure FASTAPI_LOG_PATH is available for the template
    final_success_template = success_template.format(url=url_to_display, fastapi_log_path=fastapi_log_path_for_template)
    final_failure_template = failure_template.format(fastapi_log_path=fastapi_log_path_for_template)
    if in_colab_flag: 
        clear_output_func(wait=True) # Clear previous print statements from this cell for final display
    if url_to_display:
        display_func(HTML_class(final_success_template))
    else:
        display_func(HTML_class(final_failure_template))

try:
    print("[2/2] 準備啟動 Streamlit 服務...") # This is part of Stage 3
    cmd_streamlit = [sys.executable, "-m", "streamlit", "run", STREAMLIT_APP_PATH, "--server.port", str(STREAMLIT_PORT), "--server.headless", "true", "--server.enableCORS", "false", "--server.enableXsrfProtection", "false"]

    streamlit_process = subprocess.Popen(cmd_streamlit, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8', cwd=PROJECT_DIR_TO_USE)
    print(f"    ✅ Streamlit 啟動指令已送出 (Port: {STREAMLIT_PORT})。")
    print("    ⏳ 等待 Streamlit 伺服器初始化並嘗試獲取連結...")

    def stream_watcher(identifier, stream):
        for line in iter(stream.readline, ''):
            if not line:
                break
            print(f"[{identifier}] {line}", end='', flush=True)
        stream.close()

    monitor_thread = threading.Thread(target=stream_watcher, args=('Streamlit', streamlit_process.stdout))
    monitor_thread.daemon = True
    monitor_thread.start()

    current_streamlit_url = None
    if IN_COLAB:
        print("\n✅ Streamlit 伺服器已在背景啟動 (streamlit_process potentially running).")
        print("   ⏳ 建議等待約 15-30 秒讓 Streamlit 服務完成初始化...")
        try:
            # Wait a bit for Streamlit to initialize before attempting to get URL
            time.sleep(15) 
            print("   繼續嘗試獲取 Streamlit URL...")
        except KeyboardInterrupt:
            print("\n⌨️ 操作被使用者中斷。")
            raise 
        except Exception as e:
            print(f"\n⚠️ 等待或輸入時發生錯誤: {e}。繼續嘗試獲取URL。")
        
        current_streamlit_url = fetch_url_with_eval_js(STREAMLIT_PORT, EVAL_JS_MAX_RETRIES, EVAL_JS_RETRY_DELAY, eval_js, time, print, IN_COLAB)
        if not current_streamlit_url:
             print("\n   ➡️ eval_js method failed for Streamlit. Will try cloudflared as a fallback if necessary.")

    if not current_streamlit_url:
        current_streamlit_url = fetch_url_with_cloudflared(STREAMLIT_PORT, CLOUDFLARED_TIMEOUT_SECONDS, subprocess, os, threading, time, print)

    # Display final status
    # The clear_output(wait=True) inside display_result_html will clear previous prints from this cell.
    display_result_html(current_streamlit_url, SUCCESS_HTML_TEMPLATE_STR, FAILURE_HTML_TEMPLATE_STR, display, HTML, clear_output, IN_COLAB, print, FASTAPI_LOG_PATH)
    
    print("\n====================================================================================")
    print(f"🚀 FastAPI (Backend) should be running at: http://localhost:{FASTAPI_PORT} (within Colab)")
    print(f"   FastAPI Logs are at: {FASTAPI_LOG_PATH} (Use next cell to view)")
    print(f"🔗 Streamlit (Frontend) Public URL (if successful): {current_streamlit_url or 'Not available - check logs'}")
    print("====================================================================================\n")

    print("--- Streamlit 應用程式即時輸出 (日誌與錯誤) ---")
    # monitor_thread.join() will block here, showing live Streamlit output 
    # until Streamlit process stops or Colab cell is interrupted.
    if monitor_thread and monitor_thread.is_alive():
        monitor_thread.join()

except KeyboardInterrupt:
    print("\n\n⌨️ 偵測到手動中斷 (KeyboardInterrupt)。")
except Exception as e:
    print(f"\n\n❌ 腳本執行期間發生主要錯誤: {e}")
    import traceback
    traceback.print_exc() 
finally:
    print("\n\n--- 正在終止服務 (Streamlit and FastAPI attempts) ---")
    if streamlit_process and streamlit_process.poll() is None:
        print("    Terminating Streamlit process...")
        streamlit_process.terminate()
        try:
            streamlit_process.wait(timeout=10) 
            print("    Streamlit process terminated.")
        except subprocess.TimeoutExpired:
            print("    Streamlit process did not terminate gracefully, killing.")
            streamlit_process.kill()
    else:
        print("    Streamlit process already terminated or not started.")
    
    print("    Attempting to terminate FastAPI background process group...")
    try:
        if fastapi_process_group_launcher and fastapi_process_group_launcher.poll() is None:
            print(f"    Sending SIGTERM to FastAPI process group (PGID: {fastapi_process_group_launcher.pid})...")
            os.killpg(fastapi_process_group_launcher.pid, signal.SIGTERM)
            time.sleep(3) # Give it a moment to terminate
            if fastapi_process_group_launcher.poll() is None: # If still running, try SIGKILL
                print("    FastAPI process group did not terminate with SIGTERM, sending SIGKILL...")
                os.killpg(fastapi_process_group_launcher.pid, signal.SIGKILL)
                time.sleep(1)
            print(f"    FastAPI process group (PGID: {fastapi_process_group_launcher.pid}) termination signal sent. Status: {fastapi_process_group_launcher.poll()}")
        else:
            print("    FastAPI process group likely already terminated or was not started properly.")
    except Exception as e_kill_fastapi:
        print(f"    Error during FastAPI process group termination: {e_kill_fastapi}")
        print("    FastAPI might still be running. It should terminate when the Colab session ends.")

    if monitor_thread and monitor_thread.is_alive():
        print("    Joining Streamlit monitor thread (if still alive)...")
        monitor_thread.join(timeout=2) 
    print("--- 腳本執行完畢 ---")


In [ ]:
# This cell allows you to view the FastAPI log file.
# Make sure the main cell above has been run first to define FASTAPI_LOG_PATH.
if 'FASTAPI_LOG_PATH' in locals() or 'FASTAPI_LOG_PATH' in globals():
    print(f"Displaying contents of FastAPI log file: {FASTAPI_LOG_PATH}\n")
    try:
        with open(FASTAPI_LOG_PATH, 'r') as f:
            log_content = f.read()
        if log_content.strip():
            print("--- FastAPI Log Start ---")
            print(log_content)
            print("--- FastAPI Log End ---")
        else:
            print(f"FastAPI log file is empty. The service might be starting, have no output, or encountered an issue.")
    except FileNotFoundError:
        print(f"FastAPI log file not found at {FASTAPI_LOG_PATH}. \nEnsure the FastAPI service started correctly and the path is correct (run main cell first).")
    except Exception as e:
        print(f"An error occurred while reading the FastAPI log: {e}")
else:
    print("FASTAPI_LOG_PATH is not defined. Please run the main setup cell above first.")

In [ ]:
# 請將從 README.md 複製的「綜合健康度檢測腳本」貼在此處並執行。
# 這個腳本將幫助您檢查 Colab 環境、API 金鑰、以及外部服務連線狀態。